In [1]:
%matplotlib inline
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

In [2]:
Train_Data = pd.read_csv("FINAL_UPDATED_8_FEATURES.csv", sep=",", engine="python")
#Train_Data = pd.read_csv("../PU_Input.csv", sep=",", engine="python")
Train_Data.columns

Index(['lncrna', 'mrna', 'lncrna-length', 'mrna-length', 'lncrna-GC%',
       'mrna-GC%', 'lncrna-rel_acc_len_ratio', 'mrna-rel_acc_len_ratio',
       'hybrid_energy(kcal/mol)', 'lncrna_rel_GC_ratio', 'Class_label'],
      dtype='object')

In [3]:
rem_cols=['lncrna', 'mrna']
Train_Data.drop(rem_cols,inplace=True,axis=1) 
Train_Data.shape

(131553, 9)

In [4]:
Test_Data_SVM=Train_Data[1675:]
Test_Data_IF=Train_Data[1675:]

In [5]:
Train_np=Train_Data.values
X_Train=Train_np[:1675,:-1]
X_Test=Train_np[1675:,:-1]
Y_Train=Train_np[:1675,-1]
X_Train.shape,Y_Train.shape,X_Test.shape

((1675, 8), (1675,), (129878, 8))

In [6]:
scaler = StandardScaler()
X_norm_train=scaler.fit_transform(X_Train)
X_norm_test=scaler.fit_transform(X_Test)



In [7]:
x_train, x_val, y_train, y_val = train_test_split(X_norm_train, Y_Train, test_size=.4,random_state=0)
x_train.shape,x_val.shape

((1005, 8), (670, 8))

In [9]:
oc_svm_clf_full = OneClassSVM(kernel='rbf', nu=0.012,random_state=0)
oc_svm_clf_full.fit(x_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\classes.py:1177: DeprecationWarning: The random_state parameter is deprecated and will be removed in version 0.22.
  " be removed in version 0.22.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto_deprecated',
      kernel='rbf', max_iter=-1, nu=0.012, random_state=0, shrinking=True,
      tol=0.001, verbose=False)

In [10]:
Train_Pred=oc_svm_clf_full.predict(x_train)
Val_Pred=oc_svm_clf_full.predict(x_val)
Train_Pred[Train_Pred==1].shape,Val_Pred[Val_Pred==1].shape



((974,), (628,))

In [11]:
if_clf = IsolationForest(contamination=0.012, max_features=7, max_samples=1.0, n_estimators=100,random_state=0)
if_clf.fit(x_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


IsolationForest(behaviour='old', bootstrap=False, contamination=0.012,
        max_features=7, max_samples=1.0, n_estimators=100, n_jobs=None,
        random_state=0, verbose=0)

In [12]:
Train_Pred=if_clf.predict(x_train)
Val_Pred=if_clf.predict(x_val)
Train_Pred[Train_Pred==1].shape,Val_Pred[Val_Pred==1].shape



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


((992,), (662,))

In [13]:
All_Pred_svm=oc_svm_clf_full.predict(X_norm_test)
Test_Data_SVM['Target']=All_Pred_svm
All_Pred_svm[All_Pred_svm==-1].shape,All_Pred_svm[All_Pred_svm==1].shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


((20900,), (108978,))

In [14]:
All_Pred_svm[All_Pred_svm==-1]

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

In [15]:
All_Pred_if=if_clf.predict(X_norm_test)
Test_Data_IF['Target']=All_Pred_if
All_Pred_if[All_Pred_if==-1].shape,All_Pred_if[All_Pred_if==1].shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


((3611,), (126267,))

In [16]:
Test_Data_SVM=Test_Data_SVM[Test_Data_SVM.Target==-1]
Test_Data_IF=Test_Data_IF[Test_Data_IF.Target==-1]

In [17]:
Test_Data_IF.head()

,lncrna-length,mrna-length,lncrna-GC%,mrna-GC%,lncrna-rel_acc_len_ratio,mrna-rel_acc_len_ratio,hybrid_energy(kcal/mol),lncrna_rel_GC_ratio,Class_label,Target
1740,1500,16321,0.361333,0.549109,0.176667,0.336989,-71.8,0.333313,0,-1
1885,1500,8003,0.361333,0.615894,0.176667,0.281145,-86.1,0.319744,0,-1
2105,19296,7805,0.411795,0.574888,0.242278,0.133376,-451.7,0.353776,0,-1
2115,19296,4065,0.411795,0.469373,0.242278,0.160886,-300.1,0.298649,0,-1
2119,19296,6738,0.411795,0.561146,0.242278,0.222024,-430.1,0.332174,0,-1


In [18]:
Reliable_Negative=pd.merge(Test_Data_IF,Test_Data_SVM,how='inner')
Reliable_Negative.loc[Reliable_Negative.Target==-1,'Target']=0
Reliable_Negative.drop(['Class_label'],inplace=True,axis=1) 
print(Reliable_Negative.shape)
Reliable_Negative.head()

(3487, 9)


,lncrna-length,mrna-length,lncrna-GC%,mrna-GC%,lncrna-rel_acc_len_ratio,mrna-rel_acc_len_ratio,hybrid_energy(kcal/mol),lncrna_rel_GC_ratio,Target
0,1500,16321,0.361333,0.549109,0.176667,0.336989,-71.8,0.333313,0
1,1500,8003,0.361333,0.615894,0.176667,0.281145,-86.1,0.319744,0
2,19296,7805,0.411795,0.574888,0.242278,0.133376,-451.7,0.353776,0
3,19296,4065,0.411795,0.469373,0.242278,0.160886,-300.1,0.298649,0
4,19296,6738,0.411795,0.561146,0.242278,0.222024,-430.1,0.332174,0


In [19]:
Positive=Train_Data[:1675]
Positive.rename(columns={'Class_label': 'Target'},inplace=True)

Positive.tail()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,lncrna-length,mrna-length,lncrna-GC%,mrna-GC%,lncrna-rel_acc_len_ratio,mrna-rel_acc_len_ratio,hybrid_energy(kcal/mol),lncrna_rel_GC_ratio,Target
1670,1733,4127,0.473745,0.431791,0.196769,0.215411,-80.4,0.271308,1
1671,9027,4127,0.387504,0.431791,0.184004,0.215411,-298.7,0.287054,1
1672,1989,4127,0.449975,0.431791,0.177979,0.215411,-98.5,0.315878,1
1673,1852,4127,0.444924,0.431791,0.179266,0.215411,-93.3,0.305047,1
1674,2111,4127,0.481762,0.431791,0.168167,0.215411,-109.7,0.336095,1


In [20]:
Full_Data=pd.concat([Positive,Reliable_Negative])
Full_Data=Full_Data.sample(frac=1,random_state=201).reset_index(drop=True)
Full_Data.head()

,lncrna-length,mrna-length,lncrna-GC%,mrna-GC%,lncrna-rel_acc_len_ratio,mrna-rel_acc_len_ratio,hybrid_energy(kcal/mol),lncrna_rel_GC_ratio,Target
0,424,12317,0.356132,0.554599,0.209906,0.158561,-37.9,0.200717,0
1,8302,874,0.408335,0.607551,0.186943,0.074371,-46.8,0.286882,1
2,3442,509,0.369843,0.493124,0.173155,0.341847,-52.1,0.390244,0
3,22743,3122,0.440531,0.475657,0.187662,0.167201,-294.6,0.350796,1
4,5822,2154,0.588801,0.409935,0.126417,0.167595,-123.4,0.463977,0


In [20]:
Full_Data.to_csv('FINAL_UPDATED_BINARY_TRAINING.csv',index=False)

In [21]:
Full_Data.shape

(5162, 9)